# Spark first

## Initialization. import spark dependencies

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)

## Disable logging

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


## Create spark session

In [3]:
import org.apache.spark.sql._

//jupyter notebook way to create spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._

//jupyter notebook way to create spark session

spark: SparkSession = org.apache.spark.sql.SparkSession@1e23547

## get sparkContext obj

In [4]:
def sc = spark.sparkContext

defined function sc

## Create RDD

In [5]:
//create RDD from collection
val rdd1 = sc.parallelize(1 to 100)
rdd1.getClass

//create RDD from text file
val txtRdd = sc.textFile("/home/pi/spark-2.4.5-bin-hadoop2.6/README.md")
txtRdd.getClass

rdd1: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd4.sc:1
res4_1: Class[T] = class org.apache.spark.rdd.ParallelCollectionRDD
txtRdd: org.apache.spark.rdd.RDD[String] = /home/pi/spark-2.4.5-bin-hadoop2.6/README.md MapPartitionsRDD[2] at textFile at cmd4.sc:5
res4_3: Class[T] = class org.apache.spark.rdd.MapPartitionsRDD

## Create Pair RDD

In [6]:
//create it from Map:
val mapScore = Map("name1"->1, "name2"->3,"name3"->2)
val nameRdd = sc.parallelize(mapScore.toSeq)

val mapAge = Map("name1"->33, "name2"->29,"name3"->45)
val ageRdd = sc.parallelize(mapAge.toSeq)

//create it from list
val mapStatus = List(("name1","good"),("name2","better"),("name3","intermediate"))
val statusRdd = sc.parallelize(mapStatus)

mapScore: Map[String, Int] = Map("name1" -> 1, "name2" -> 3, "name3" -> 2)
nameRdd: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[3] at parallelize at cmd5.sc:2
mapAge: Map[String, Int] = Map("name1" -> 33, "name2" -> 29, "name3" -> 45)
ageRdd: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[4] at parallelize at cmd5.sc:5
mapStatus: List[(String, String)] = List(
  ("name1", "good"),
  ("name2", "better"),
  ("name3", "intermediate")
)
statusRdd: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[5] at parallelize at cmd5.sc:9

## transforming of RDD

### transforming - 1

In [7]:
//common transformations supported by RDD
//filter
val rddLessThan10 = rdd1.filter(item=>item<10)

//find line with max word count
//map
val txtRddLineSize = txtRdd.map(line=>line.split(" ").size)

//flatMap. will break each line into multiple items
val pairMap = txtRdd.flatMap(line => line.split(" ")).map(word => (word,1))
pairMap.take(4)

take at cmd6.sc:9

1 / 1

rddLessThan10: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[6] at filter at cmd6.sc:1
txtRddLineSize: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[7] at map at cmd6.sc:5
pairMap: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[9] at map at cmd6.sc:8
res6_3: Array[(String, Int)] = Array(
  ("#", 1),
  ("Apache", 1),
  ("Spark", 1),
  ("", 1)
)

### transforming - 2

In [8]:
//groupByKey. when called on dataset of (K,V) pairs, returns a dataset of (K,Iterable<V>) pairs
val groupByKeyMap = pairMap.groupByKey()
groupByKeyMap.take(5)

//reduceByKey. when called on dataset of (K,V) pairs, returns a dataset of (K,V) pairs
//where V for each K are aggregated by giving reduce func.
val reduceByKeyMap = pairMap.reduceByKey((a,b)=>a+b)
reduceByKeyMap.take(5)

//aggregateByKey

//sortByKey. first parameter indicating sorting by asc or not
val maxWordCountMap = reduceByKeyMap.map(k=>(k._2,k._1)).sortByKey(false)
maxWordCountMap.take(5)

//join
val nameAgeRdd = nameRdd.join(ageRdd)
nameAgeRdd.collect()

val nameAgeCogroupRdd = nameRdd.cogroup(ageRdd)
nameAgeCogroupRdd.collect()

map at cmd6.sc:8

2 / 2

take at cmd7.sc:2

1 / 1

map at cmd6.sc:8

...

parallelize at cmd5.sc:5

0 / 4

collect at cmd7.sc:20 
 (kill)

0 / 4

groupByKeyMap: org.apache.spark.rdd.RDD[(String, Iterable[Int])] = ShuffledRDD[10] at groupByKey at cmd7.sc:1
res7_1: Array[(String, Iterable[Int])] = Array(
  ("package", CompactBuffer(1)),
  ("this", CompactBuffer(1)),
  ("integration", CompactBuffer(1)),
  ("page](http://spark.apache.org/documentation.html).", CompactBuffer(1)),
  ("Python", CompactBuffer(1, 1))
)
reduceByKeyMap: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at reduceByKey at cmd7.sc:6
res7_3: Array[(String, Int)] = Array(
  ("package", 1),
  ("this", 1),
  ("integration", 1),
  ("Python", 2),
  ("page](http://spark.apache.org/documentation.html).", 1)
)
maxWordCountMap: org.apache.spark.rdd.RDD[(Int, String)] = ShuffledRDD[15] at sortByKey at cmd7.sc:12
res7_5: Array[(Int, String)] = Array(
  (72, ""),
  (23, "the"),
  (17, "to"),
  (15, "Spark"),
  (12, "for")
)
nameAgeRdd: org.apache.spark.rdd.RDD[(String, (Int, Int))] = MapPartitionsRDD[18] at join at cmd7.sc:16
res7_7: Array[(String, (Int, Int))] = 

### transforming - 3

In [9]:
//create it from list
val idStatus = List(("id1","1"),("id2","1"),("id1","0"))
val idStatusRdd = sc.parallelize(idStatus)

idStatus: List[(String, String)] = List(
  ("id1", "1"),
  ("id2", "1"),
  ("id1", "0")
)
idStatusRdd: org.apache.spark.rdd.RDD[(String, String)] = ParallelCollectionRDD[21] at parallelize at cmd8.sc:2

In [10]:
//def checkValue(arg1:String, arg2:String):String={
//    return "twoValue"
//}

//print(checkValue("abc","ddd"))

def checkTuple(arg: Tuple2[String,String]):Int={
    arg._2 match{
        case "1"=>1
        case "0"=>1
        case _=>0
    }
}
println(checkTuple(("id1","p")))
println(checkTuple(("id1","1")))
idStatusRdd.map(a=>(a._1,checkTuple(a))).collect()

/*def checkValue(arg: Iterable[String]):Int={
    var count = 0
    val it = arg.iterator
    while(it.hasNext){
        count = count + 1
    }
    return count
}

//val idStatusRddReduced = idStatusRdd.reduceByKey((x,y)=>checkValue(x,y))
val idStatusRddReduced = idStatusRdd.groupByKey().reduceByKey(checkValue)
idStatusRddReduced.collect()
*/

0
1


collect at cmd9.sc:10

4 / 4

defined function checkTuple
res9_3: Array[(String, Int)] = Array(("id1", 1), ("id2", 1), ("id1", 1))

## Actions of RDD

In [11]:
//collect will display RDD items
rddLessThan10.collect()

//count. show txtRdd size
txtRdd.count()

//take
txtRddLineSize.take(3)
val maxLineWordCount = txtRddLineSize.reduce((a,b) => if (a>b) a else b)
val lineWithMaxWordCount = txtRdd.filter(line => line.size == 16)
lineWithMaxWordCount.collect()

//contByKey
var rdd1 = sc.makeRDD(Array(("A",0),("A",2),("B",1),("B",2),("B",3)))
rdd1.getClass
rdd1.countByKey()

//countByKey can't be called on type class org.apache.spark.rdd.MapPartitionsRDD
pairMap.getClass
//val countByKeyMap = pairMap.countyByKey()
//countByKeyMap.take(5)

collect at cmd10.sc:1

4 / 4

count at cmd10.sc:4

2 / 2

take at cmd10.sc:7

...

countByKey at cmd10.sc:17

4 / 4

countByKey at cmd10.sc:17

4 / 4

res10_0: Array[Int] = Array(1, 2, 3, 4, 5, 6, 7, 8, 9)
res10_1: Long = 104L
res10_2: Array[Int] = Array(3, 1, 14)
maxLineWordCount: Int = 16
lineWithMaxWordCount: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[23] at filter at cmd10.sc:9
res10_5: Array[String] = Array("## Running Tests", "## Configuration")
rdd1: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[24] at makeRDD at cmd10.sc:13
res10_7: Class[T] = class org.apache.spark.rdd.ParallelCollectionRDD
res10_8: collection.Map[String, Long] = Map("A" -> 2L, "B" -> 3L)
res10_9: Class[T] = class org.apache.spark.rdd.MapPartitionsRDD

## creating Dataset

In [12]:
val dataset = spark.read.textFile("/home/pi/spark-2.4.5-bin-hadoop2.6/README.md")

dataset: Dataset[String] = [value: string]

In [13]:
dataset.count()
dataset.first()

count at cmd12.sc:1

1 / 1

count at cmd12.sc:1

1 / 1

first at cmd12.sc:2

1 / 1

res12_0: Long = 104L
res12_1: String = "# Apache Spark"

In [14]:
val linesWithSpark = dataset.filter(txt=>txt.contains("Spark"))
linesWithSpark.count()


count at cmd13.sc:2

1 / 1

count at cmd13.sc:2

1 / 1

linesWithSpark: Dataset[String] = [value: string]
res13_1: Long = 19L

### find the line with most words

In [20]:
//https://spark.apache.org/docs/latest/quick-start.html example can't run here:
dataset.getClass
//dataset.map(line => line.split(" ").size)

val rddx = sc.textFile("hdfs://localhost:9000/user/pi/input/core-site.xml")

res19_0: Class[T] = class org.apache.spark.sql.Dataset
rddx: org.apache.spark.rdd.RDD[String] = hdfs://localhost:9000/user/pi/input/core-site.xml MapPartitionsRDD[48] at textFile at cmd19.sc:4

In [21]:
rddx.take(2)

take at cmd20.sc:1

1 / 1

res20: Array[String] = Array(
  "<?xml version=\"1.0\" encoding=\"UTF-8\"?>",
  "<?xml-stylesheet type=\"text/xsl\" href=\"configuration.xsl\"?>"
)